In [2]:
import tensorflow as tf
import numpy as np

In [6]:
class modified_gru():
    """
        It's a modified version of normal GRU
    """
    def __init__(self,time_steps,init_state,scalar_values,input_size,
                 output_size,final_mem,question,input_seq=None):
        
        # inputs required for sequential memory vectors e_i 
        self.time_steps = time_steps              # equal to max no concepts
        self.init_state = init_state              # initial secondary input ideally zero tensor
        self.input_seq = input_seq                # input seq to be evaluated
        self.scalar_values = scalar_values        # gated scalar values 
        self.input_size = input_size              # size of input vector 
        self.output_size = output_size            # size of output required
        
        # inputs required for Answer modules
        self.question = question
        self.final_mem = final_mem
        self.word_weight = tf.get_variable("word_w",[output_size,output_size],dtype=tf.float64)
        self.word_bias = tf.get_variable("word_b",[output_size,1],dtype=tf.float64)
#         self.init_word = init_word          # should be ideally a zero vector
        
        # parameters of gru
        self.W_z,self.W_r,self.W_h = tf.get_variable("update_w",[output_size,input_size],dtype=tf.float64),tf.get_variable("reset_w",[output_size,input_size],dtype=tf.float64),tf.get_variable("out_w",[output_size,input_size],dtype=tf.float64)
        self.U_z,self.U_r,self.U_h = tf.get_variable("update_u",[output_size,output_size],dtype=tf.float64),tf.get_variable("reset_u",[output_size,output_size],dtype=tf.float64),tf.get_variable("out_u",[output_size,output_size],dtype=tf.float64)
        self.b_z,self.b_r,self.b_h = tf.get_variable("update_b",[output_size,1],dtype=tf.float64),tf.get_variable("reset_b",[output_size,1],dtype=tf.float64),tf.get_variable("out_b",[output_size,1],dtype=tf.float64)
    """
    right now its a normal gru !!!
    You need to change the the output !!!
    """
    def mod_gru_unit(self,h_prev,steps,out=[]):
        index = steps - self.time_steps 
        input_vector = self.input_seq[index]
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z,name="matmul_2")
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        h_t = (self.scalar_values[index]*h_t) + ((1-self.scalar_values[index])*h_prev)
        out.append(h_t)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.mod_gru_unit(h_prev = h_t,out = out,steps = steps) 
        
    def answer_gru_unit(self,h_prev,word_prev,steps,out=[]):
        index = steps - self.time_steps 
#         print(np.shape(word_prev),np.shape(self.question))
        input_vector = tf.concat([word_prev,self.question],axis=0)
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector,name = 'matmul_1')+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        word_pred = tf.nn.softmax(tf.matmul(self.word_weight,h_t)+self.word_bias)
#         print(h_t)
#         print(word_pred)
        out.append(word_pred)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.answer_gru_unit(h_prev = h_t,word_prev = word_pred,out = out,steps = steps) 
    
       

In [7]:
tf.reset_default_graph()
gru = modified_gru(time_steps = 10,
                   init_state = np.random.random(size=(20,1)),
                   input_seq = np.random.random(size=(10,40,1)),
                   scalar_values = np.random.random(size=(10,1)),
                   input_size = 40,
                   output_size = 20,
                   question = np.random.random(size=(20,1)),
                   final_mem = np.random.random(size=(20,1)))
gru.answer_gru_unit(h_prev=np.random.random(size=(20,1)),
                    word_prev = gru.init_state,
                    steps = gru.time_steps)

[<tf.Tensor 'Softmax:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_1:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_2:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_3:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_4:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_5:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_6:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_7:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_8:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'Softmax_9:0' shape=(20, 1) dtype=float64>]

In [8]:
tf.reset_default_graph()
gru = modified_gru(time_steps = 10,
                   init_state = np.random.random(size=(20,1)),
                   input_seq = np.random.random(size=(10,20,1)),
                   scalar_values = np.random.random(size=(10,1)),
                   input_size = 20,
                   output_size = 20,
                   question = np.random.random(size=(20,1)),
                   final_mem = np.random.random(size=(20,1)))
gru.mod_gru_unit(steps = gru.time_steps,h_prev=gru.init_state)

[<tf.Tensor 'add_7:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_15:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_23:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_31:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_39:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_47:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_55:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_63:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_71:0' shape=(20, 1) dtype=float64>,
 <tf.Tensor 'add_79:0' shape=(20, 1) dtype=float64>]